In [1]:
import pandas as pd
from beda import *

### Чтение данных из bed-разметок

#### Информация о промоторах в аннотации TAIR.

In [2]:
markup_tair = pd.read_csv("markup_data/ups2000_tair10.bed", sep="\t", header=None)

In [3]:
markup_tair = markup_tair.loc[(markup_tair[0] == 'chr1') & (markup_tair[3] == '+')]

In [4]:
markup_tair.head()

,0,1,2,3,4
0,chr1,1631,3631,+,AT1G01010
3,chr1,21146,23146,+,AT1G01040
7,chr1,42677,44677,+,AT1G01073
10,chr1,50239,52239,+,AT1G01110
12,chr1,54624,56624,+,AT1G01115


#### Информация о пиках ATAC-seq.

In [5]:
markup_ATAC = pd.read_csv("markup_data/GSM2704255_ATAC_chr1.bed", sep="\t", header=None)

In [6]:
markup_ATAC.head()

,0,1,2,3,4
0,chr1,33153,33510,+,1
1,chr1,37877,38336,+,2
2,chr1,55331,55668,+,3
3,chr1,57011,57250,+,4
4,chr1,88271,88615,+,5


### Использование конвейера

#### Проверка на наличие самопересечений внутри одной разметки.

In [7]:
is_slf_intrsct(markup_tair)

  0%|▏                                                                            | 10/3557 [00:00<00:00, 10031.82it/s]


True

In [8]:
is_slf_intrsct(markup_ATAC)

100%|█████████████████████████████████████████████████████████████████████████| 6195/6195 [00:00<00:00, 3076087.76it/s]


False

#### Чтение данных о длинах хромосом.

In [9]:
chromo_len = pd.read_csv('markup_data/chr_lengths.txt', sep="\t", header=None)

In [10]:
chromo_len[0][0]

30427671

#### Поиск пересечений между разметками. Попарный анализ всех существующих областей.

In [11]:
markup_intrscts = build_intrscts(markup_tair, markup_ATAC)

100%|█████████████████████████████████████████████████████████████████████████████| 3558/3558 [00:14<00:00, 248.58it/s]


#### Деление хромосомы на целое количество отрезков.

In [12]:
markup_intrscts = segmentation_intrsct(markup_tair, markup_ATAC,
                                       chr_length=chromo_len[0][0], n_segments=10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.64it/s]


In [13]:
markup_intrscts = segmentation_intrsct(markup_tair, markup_ATAC,
                                       chr_length=chromo_len[0][0], n_segments=100)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 52.69it/s]


#### Сохранение данных.

In [14]:
markup_intrscts.to_csv('markup_data/TAIR10_vs_ATAC.bed', header=None, index=None, sep='\t')